# Witness rankについて

参考：
* [Model-based RL in Contextual Decision Processes: PAC bounds and Exponential Improvements over Model-free Approaches](https://arxiv.org/abs/1811.08540)

[Bellman rank](RL_Bellman_rank.ipynb)の続きです．
Witness rankはモデルベースRL向けのBellman rankみたいなものです．
モデルベースとモデルフリーでは一般に，
* モデルベース：環境のダイナミクスをモデル化し，それを解くことで最適方策を見つける　
* モデルフリー：最適方策やその価値を直接モデル化する

といった違いがあります．
モデルベースはモデルフリーよりも一般にサンプル効率が良いことが信じられています（教師ありの情報をたくさん使えるので）．

~~TODO: globalな探索が重要な環境ではモデルフリーでしか解けないことがBellman rankの論文で指摘されているっぽい．要確認．~~
（これはWitness rankが提案された時点ではBellman rankを解くアルゴリズムがモデルフリーしか提案されてないだけかも）

今回は，関数近似が十分リッチであれば，モデルベースな手法がモデルフリーよりもサンプル効率が指数的に良いことを示していきます．

表記：

Contextual decision process (CDP)を考えます．

* $\mathcal{X}$を$\mathcal{X}_1, \dots, \mathcal{X}_{H+1}$に分割可能な文脈の空間とします．
* $|\mathcal{A}|=K$とします．

---

Contextual MDPの重要な例として，Factored MDPがあります．

* $d\in \mathbb{N}$と小さな有限集合$\mathcal{O}$に対して，文脈空間を$\mathcal{X} = [H] \times \mathcal{O}^d$とします．つまり，何らかのベクトルが状態に対応している状況ですね（Linear MDPみたいな処理かも？）．
* 状態$x$とインデックス$i \in [d]$ について，$x[i]$を$x$における$i$番目の「状態の値」とします．
* それぞれの状態の値$i \in [d]$について，$\mathrm{pa}_i \subseteq[d]$を$i$の「親」と呼びます．この$\mathrm{pa}_i$が$i$に直接影響を与えます．

このとき，Factored MDPでは，

$$
\forall h, x \in \mathcal{X}_h, x^{\prime} \in \mathcal{X}_{h+1}, a \in \mathcal{A}, P\left(x^{\prime} \mid x, a\right)=\prod_{i=1}^d P^{(i)}\left[x^{\prime}[i] \mid x\left[\mathrm{pa}_i\right], a, h\right]
$$

の形式で遷移確率が分解されます．

このとき，遷移確率は
* $\sum^{d}_{i=1} HK \cdot |\mathcal{O}|^{1+|\operatorname{pa}_i|}$個のパラメータで構成されます．
* 一方で，factoredではないMDPでは$d H K |\mathcal{O}|^{1+d}$のパラメータがかかることに注意しましょう（つまり$\mathrm{pa}_i$の項がありません）．
よって，$\mathrm{pa}_i$次第では，factored MDPの方が遷移に必要なパラメータの数が少ないです．

---

* モデルベースでは報酬と遷移のモデル$M=(R, P)$を$M \in \mathcal{M}$から選択して，最も良いものを見つけるとします．
    * $(r, x') \sim M_{x, a}$は$r\sim R(x, a)$かつ$x' \sim P_{x, a}$として表記します
    * $\operatorname{OP}(M)$で，モデル$M$についての最適価値と最適方策を計算するオラクルを表すとします．

## モデルベースとモデルフリーの定義

よく使われるモデルフリーの定義は，「メモリのスペースが$o\left(|\mathcal{X}|^2|\mathcal{A}|\right)$であるアルゴリズム」ですが，
これは関数近似がある場合にスケールしません．

そこで，次の情報理論的な定義を考えましょう．

---

* （有限な）関数集合$\mathcal{G}: (\mathcal{X} \times \mathcal{A}) \to \mathbb{R}$を考えます．これは例えばQ関数や方策のクラスに相当します．
* 状態$x$を受け取ると，$|\mathcal{G}|$と$a$上のベクトルを返すような，$\Phi_{\mathcal{G}}(x):=[g(x, a)]_{g \in \mathcal{G}, a \in \mathcal{A}}$なるマッピングを考えます．この返ってきたベクトルを$\mathcal{G}$-profileと呼ぶことにします．

次を満たす時，「アルゴリズムは$\mathcal{G}$を使ったモデルフリーアルゴリズムである」といいます．

**全ての$x\in \mathcal{X}$に，$\Phi_{\mathcal{G}}(x)$を使ってアクセスする．**

---

つまり，モデルフリーでは$x$と$x'$の違いを，$\Phi_{\mathcal{G}}(x)$と$\Phi_{\mathcal{G}}(x')$を使ってのみ認識できます．
（例えば$x$と$x'$の違いを$Q(x, \cdot)$と$Q(x', \cdot)$を使って判別する，など）

* **コメント**：モデルベースの場合は関数集合$\mathcal{G}: (\mathcal{X} \times \mathcal{A}) \to \mathbb{R}^{\mathcal{X}}$によってアクセスしてる気がする．この場合は$\mathcal{G}$が$P$のクラスに対応するはずであり，環境がどう動くかわからないならば，$\mathcal{G}$には指示関数であるような$P$が含まれるはず．その場合は明らかに情報の損失が存在しないため，モデルベースでは情報の損失が発生しない，と言えそう．ただこれを数学的に記述するのは結構むずそう．今回の定義も正直微妙だと思っている．

そのため，一般には何らかの情報理論的な損失が発生します．
しかし次のように，Tabular MDPにおいては，モデルベースとモデルフリーアルゴリズムは情報理論的には同じになります．

---

**Tabular MDPについて**

テーブル形式のMDPでは$\mathcal{X}$と$\mathcal{A}$が有限集合になります．また，よくあるQ関数のクラスとして，$\mathcal{Q}=\{Q: \mathcal{X} \times \mathcal{A} \rightarrow[0,1]\}$を考えます（これを考えれば，どんなTabular MDPのクラスを考えても最適Q関数がここに入っています）．
最後に，$\mathcal{G}=\mathcal{Q}$としましょう．

各状態$x\in\mathcal{X}$について，$Q^x\left(x^{\prime}, a\right)=\mathbf{1}\left\{x=x^{\prime}\right\}$なる関数を考えましょう．
このとき，明らかに$Q^x \in \mathcal{Q}, \forall x \in \mathcal{X}$が成立します．

任意の状態$x' \in \mathcal{X}$について，$\Phi_{\mathcal{Q}}\left(x^{\prime}\right)$は$\mathcal{Q}$と$\mathcal{A}$上の関数でした．
任意の$x \in \mathcal{X}$に対して$Q^x \in \mathcal{Q}$なので，$\left[Q^x\left(x^{\prime}, a\right)\right]_{x \in \mathcal{X}}$は$\Phi_{\mathcal{Q}}\left(x^{\prime}\right)$の部分ベクトルになります．
また，$x'=x$となる$x$でだけ$Q^x$は値が１になるので，$\Phi_{\mathcal{Q}}\left(x^{\prime}\right)$を調べることで$x'$を復元することができます．

まとめると，$x \mapsto \Phi_{\mathcal{Q}}(x)$はテーブル形式の場合はInvertibleであり，情報の損失が発生しません．

---

一方で，関数近似がある場合にはモデルフリーでは解けないMDPが存在します．

---

**定理**

次を満たすCDPの族$\mathcal{M}$が存在します．

* $|\mathcal{M}|\leq 2^H$
* 確率$1- \delta$以上で，$\mathcal{M}$を使ったモデルベースアルゴリズム（Factored MDP用のAlgorithm 1）が$\epsilon$-最適な方策を$\operatorname{poly}(H, 1 / \epsilon, \log (1 / \delta))$個の軌跡で吐き出す
* $\mathcal{G}=O P(\mathcal{M})$として，どんなモデルフリーアルゴリズムも$o(2^H)$の軌跡を使っても$\epsilon$-最適方策が吐き出せない 

**ムズMDPの構築**

〜簡単な流れ〜

* まず，２-ary treeっぽいFactored MDPを作ります．
* 続いてこのFactored MDPがWitness rank用のアルゴリズムで解けることを示します
* このFactored MDPに似ていますが，どのアルゴリズムを使ってもサンプル効率が$2^H$になるNon-factoredなMDPを作ります
* 最後に，モデルフリーなアルゴリズムはこのnon-factoredなMDPとfactored MDPが区別できないことを示します．よって，モデルフリーアルゴリズムはFactored MDPでもサンプル効率が$2^H$になります．

順番にやっていきましょう．次のFactored MDPを考えます

![factoredMDP](figs/hard-factored-MDP.png)

$d > 2$について，$H=d+2$とします．
Factored MDPでは各状態が$d$次元ベクトルに相当することを思い出しましょう．ここでは，状態の値は$\mathcal{O}=\{-1, 0, 1, 2\}$から値を取ることにします（次に見るように，追加でステップ数もベクトルに記載されます）．

* 状態空間は$\mathcal{X}=[H] \times \mathcal{O}^d$とし，$\mathcal{A}=\{-1, +1\}$とします．
* 初期状態: $x=1 \circ[0]^d$とします．ここで$[0]^d$は要素がすべて$0$の$d$次元ベクトルであり，$\circ$はベクトルの結合を表します．
* 要素数が$2^d$のモデルの集合を考えます（どのモデルが選択されているかはエージェントにはわかりません）．
  * それぞれのモデルは長さ$d$の行動の系列$\mathbf{p}=\left\{p_1, \ldots, p_d\right\}$で判別されます．ここで，$p_i \in \{-1, 1\}$です．
  * この$\mathbf{p}$は遷移確率にだけ影響するので，各モデルを$P^\mathbf{p}$で表すことにします．（報酬は同じ＆既知であるとします）
  * つまり，モデルの集合は$\{\mathbf{p} \text{の全パターン}\}$であり，$2^d$個あります．
* Factored MDPでは状態の値の$i$番目の親が$i$番目の値に影響します．今回は$i$の親は$i$自身であるとします．つまり，$1+d$個ある状態の値はそれぞれ独立しています．
* また，遷移は決定的とします．つまり，状態$(h, x_1, \dots, x_d)$で行動$a$を取ると，次状態は$\left(h+1, P_h^{\mathbf{p}, 1}\left(x_1, a\right), \ldots, P_h^{\mathbf{p}, d}\left(x_d, a\right)\right)$に遷移します．

**$h \leq H-1$までの遷移**

このFactored MDPにおいて，$H-1$までの遷移は完全２分木を構成します．各状態には次のように，今まで取ってきた行動の値が記録されていきます（図を見るとわかりやすいかも）．

$$
\begin{aligned}
& P_h^{\mathbf{p}, i}(v, a)=v, \quad \forall v \in \mathcal{O}, a \in \mathcal{A}, i \neq h \\
& P_h^{\mathbf{p}, i}(v, a)=a, \quad \forall v \in \mathcal{O}, a \in \mathcal{A}, i=h
\end{aligned}
$$

例えばステップ$h=1$で$a=-1$を取ると，状態の値は$\cdot, -1, \cdots$として記録され，それが下層に伝搬していきます．
ここまでの遷移はモデルの集合全てについて同じですね．

**$H-1$から$H$への遷移**

最後のステップでは行動が一つしかないです．それを$+1$としましょう．
そして，
$$
P_{H-1}^{\mathbf{p}, i}\left(p_i,+1\right)=p_i, \quad \forall i \in[d], \quad \text { and } \quad P_{H-1}^{\mathbf{p}, i}\left(\bar{p}_i,+1\right)=2, \quad i \in[d]
$$
とします．
つまり，最後の層では，モデルの識別子$\mathbf{p}$と，エージェントが取った行動が異なる要素が$2$として設定されます．

また，最後の層では一つも$2$が入っていない状態のみ$+1$の報酬が発生します．
つまり，
最適方策は$\mathbf{p}$と同じ行動を取り続ける方策であり，最後の層の状態には，「今まで取った行動の何が間違っていたのか？」が記述されています．

例えば
* $\mathbf{p}=(-1, 1, -1)$
* エージェントが取った行動：$\mathbf{a}=(1, 1, 1)$

の場合，エージェントがたどり着く状態は$(5, 2, 1, 2)$になります．

このFactored MDPは，状態の要素にアクセスできる場合は簡単に解くことができます（実際，Witness rank用のアルゴリズムは多項式時間で解けます）．
モデルベースでは最後の状態の値を見ることができるので，値のうち$2$になっている箇所の行動を修正すれば，最悪でも$d$回くらいの修正で解けるわけですね．

〜激ムズMDPの構築〜

続いて，よく似ていますがUnfactoredなMDPを作ります．
ここでは各モデルは$\tilde{P}^{\mathbf{p}}$と表記します．
$H-1$までの遷移は上のFactored MDPと同じとします．

最後の層では，次のような遷移を考えます：

$$
\tilde{P}_h^{\mathbf{p}}\left(\left(H-1, x_1, \ldots, x_d\right)\right)=\left\{\begin{array}{lr}
\left(H, x_1, \ldots, x_d\right) & \text { if } x_i=\mathbf{p}_i \forall i \in[d] \\
H \circ[2]^d & \text { otherwise } .
\end{array}\right.
$$

つまり，Factored MDPでは最後の層で「今までの行動のどこが間違っていたか？」という情報が，今回は全部$2$のベクトルによって隠されています．
もちろんエージェントは$\mathbf{p}$が何かを知らないので，このMDPにおいて最適方策を当てるには，$\Omega\left(2^H\right)$のサンプルが必要になります（Csaba先生の講義におけるhay-stack lemmaってやつですね）．

〜モデルフリーアルゴリズムは激ムズMDPとムズMDPを区別できない〜

最後に，モデルフリーなアルゴリズムは上の激ムズMDPとムズMDPが区別できないことを示しましょう．

* ムズMDPの集合$\mathcal{M}=\left\{P^{\mathbf{p}}\right\}_{\mathbf{p} \in\{-1,1\}^d}$ 
  * ムズMDPの集合$\mathcal{M}$について，その最適Q関数の集合を$\mathcal{Q}$，最適方策の集合を$\Pi$とします．
* 激ムズMDPの集合$\widetilde{\mathcal{M}}=\left\{\tilde{P}^{\mathbf{p}}\right\}_{\mathbf{p} \in\{-1,1\}^d}$
  * 激ムズMDPの集合$\widetilde{\mathcal{M}}$について，その最適Q関数の集合を$\widetilde{\mathcal{Q}}$，最適方策の集合を$\widetilde{\Pi}$とします．

各$\mathbf{p}$に対応する最適方策だけ考えても，全ての枝への遷移の仕方を考えるので，この方策と$Q$関数の集合を考えるのは自然ですね．

ここで，モデルフリーは状態$x$を，$\mathcal{G}$-profileを通じてのみ観測できることを思い出しましょう．
以下，$\mathcal{M}$における$\mathcal{G}$-profileと$\widetilde{\mathcal{M}}$における$\mathcal{G}$-profileが同じになることを示します．つまり，モデルフリーアルゴリズムは，ムズMDPと激ムズMDPを区別することができません．

次を示すことで上記の目標を達成します：
* $P^\mathbf{p}$と$\widetilde{P}^\mathbf{p}$が同じ報酬の分布を持つ
* 行動の系列$\mathbf{a}$を取ったときに通過する任意の状態について，その$Q$-profileが同じになる．つまり，$\left[Q\left(x_h, \cdot\right)\right]_{Q \in \mathcal{Q}}$ と$\left[Q\left(x_h, \cdot\right)\right]_{Q \in \tilde{\mathcal{Q}}}$が同じになる．

これの証明はほぼ自明っぽいので省略します（後は読めばわかる）．

以上から，このモデルフリーアルゴリズムでは，最後に到達した状態$x$が何であるかが判別できません．
よって，ムズMDPと激ムズMDPのサンプル効率が同じになります．

---

実際，factored MDPにおいてモデルフリーアルゴリズムは上手く行かないみたいですね（lower boundはまだ出されてないみたいですが）

## Witness model misfit

モデルの誤差を定量的に評価するために，分布間の距離を測る一般的な指標である[Integral Probability Metric](https://en.wikipedia.org/wiki/Integral_probability_metric)を使います．
これは，与えられた関数クラス$\mathcal{F}$を使うと，２つの分布がどれくらい離れているかを測る指標です．
例えばWasserstein-1距離や全変動距離を一般化することができます．

---

**Integral probability metric**

$P_1, P_2 \in \Delta(\mathcal{Z})$ over $z \in \mathcal{Z}$と，Symmetric（つまり$f\in \mathcal{F}$なら$-f\in \mathcal{F}$）な関数の集合$\mathcal{F}: \mathcal{Z} \rightarrow \mathbb{R}$について，

$$
\sup _{f \in \mathcal{F}} \mathbb{E}_{z \sim P_1}[f(z)]-\mathbb{E}_{z \sim P_2}[f(z)]
$$

のことを$\mathcal{F}$のIPMと呼ぶ．

ここで，$f \in\mathcal{F}$はしばしばcriticと呼ばれ，特に$\sup$を達成する$f$は$witness$関数と呼ばれます（分布間の違いを"witness"するからだそうです）．
実際，$\sup_f$は$P_1$からのサンプルには大きな値を，$P_2$からのサンプルには小さな値を割り当てようとするので，実質$P_1$と$P_2$に対するclassifierとして機能します．

---

**Witness model misfit**

次の

* クラス：$\mathcal{F}: \mathcal{X} \times \mathcal{A} \times \mathcal{R} \times \mathcal{X} \rightarrow \mathbb{R}$：$(x, a, r, x')$を受け取って何か実数を返す関数．例えばベルマン誤差など．
* モデル：$M, M^{\prime} \in \mathcal{M}$
* タイムステップ：$h\in [H]$

について，$M'$が$M$にwitnessされたときの$h$におけるWitnessed model misfitは

$$
\mathcal{W}\left(M, M^{\prime}, h ; \mathcal{F}\right) \triangleq \sup _{f \in \mathcal{F}} \underset{\substack{x_h \sim \pi_M \\ a_h \sim \pi_{M^{\prime}}}}{\mathbb{E}}\left[\underset{\left(r, x^{\prime}\right) \sim M_h^{\prime}}{\mathbb{E}}\left[f\left(x_h, a_h, r, x^{\prime}\right)\right]-\underset{\left(r, x^{\prime}\right) \sim M_h^{\star}}{\mathbb{E}}\left[f\left(x_h, a_h, r, x^{\prime}\right)\right]\right],
$$

で定義されます．ここで，モデル$M=(R, P)$について，$\left(r, x^{\prime}\right) \sim M_h$ は $r \sim R_{x_h, a_h}, x^{\prime} \sim P_{x_h, a_h}$の略記です．

---

**直感**

* misfitは，$M'$と$M^*$によって生じる２つの分布のIPMを測っています．
    * この分布は$\mathcal{X} \times \mathcal{A} \times \mathcal{R} \times \mathcal{X}$上に定義されます．
* もちろん$M_h'=M_h^*$ならmisfitは０になります．
* $M$はデータを集めるためのモデルです．[RL_Bellman_rank.ipynb](RL_Bellman_rank.ipynb)と同じノリ？

例えば$\mathcal{F}=\left\{f:\|f\|_{\infty} \leq 1\right\}$であれば，misfitは

$$\mathcal{W}\left(M, M^{\prime}, h ; \mathcal{F}\right)=\mathbb{E}\left[\left\|R_{x_h, a_h}^{\prime} \circ P_{x_h, a_h}^{\prime}-R_{x_h, a_h}^{\star} \circ P_{x_h, a_h}^{\star}\right\|_{T V} \mid x_h \sim \pi_M, a_h \sim \pi_{M^{\prime}}\right]$$

になります（ここで$R_{x, a} \circ P_{x, a}$は$\mathcal{R}\times \mathcal{X}$上の分布です）．これは単に$R^{\prime} \circ P^{\prime}$ and $R^{\star} \circ P^{\star}$間のTV距離を測ってるだけです．

---

**Average Bellman Errorとの関係**

Bellman rankのAverage Bellman Errorを思い出しましょう．Average Bellman Errorは次で定義されます：

$$
\mathcal{E}_B\left(Q, Q^{\prime}, h\right) \triangleq \mathbb{E}\left[Q^{\prime}\left(x_h, a_h\right)-r_h-Q^{\prime}\left(x_{h+1}, a_{h+1}\right) \mid x_h \sim \pi_Q, a_{h: h+1} \sim \pi_{Q^{\prime}}\right]
$$

ここで，Q関数が$\mathcal{Q}=\mathrm{OP}(\mathcal{M})$の形をしている場合，上の定義を２つのモデル$M, M'$に対する誤差として拡張できます．
そのとき，$M, M'$に対するAverage Bellman Errorは関数$f_{M^{\prime}}\left(x, a, r, x^{\prime}\right)=r+V_{M^{\prime}}\left(x^{\prime}\right)$によってwitnessされたmisfitと同じです．

---

$\mathcal{F}$は何でも良いわけではないです．次を仮定します：
（クラス$\mathcal{F}: \mathcal{Z}\to \mathbb{R}$は$f \in \mathcal{F}$なら$-f \in \mathcal{F}$のときに対象であるといいます．）

$\mathcal{F}$は対象であり，有限なサイズとします．
$\|f \|_\infty \leq 2$とし，
$$
\forall M, M^{\prime} \in \mathcal{M}: \mathcal{W}\left(M, M^{\prime}, h ; \mathcal{F}\right) \geq \mathcal{E}_B\left(Q_M, Q_{M^{\prime}}, h\right)
$$
が成立するとします．つまり，

$$
\begin{aligned}
&\sup _{f \in \mathcal{F}} \underset{\substack{x_h \sim \pi_M \\ a_h \sim \pi_{M^{\prime}}}}{\mathbb{E}}\left[\underset{\left(r, x^{\prime}\right) \sim M_h^{\prime}}{\mathbb{E}}\left[f\left(x_h, a_h, r, x^{\prime}\right)\right]-\underset{\left(r, x^{\prime}\right) \sim M_h^{\star}}{\mathbb{E}}\left[f\left(x_h, a_h, r, x^{\prime}\right)\right]\right]
\\\geq & 
\mathbb{E}\left[Q_{M'}\left(x_h, a_h\right)-r_h-Q_{M^{\prime}}\left(x_{h+1}, a_{h+1}\right) \mid x_h \sim \pi_Q, a_{h: h+1} \sim \pi_{Q_{M'}}\right]
\end{aligned}
$$

を仮定します．

**仮定についての注意**

* この仮定は，$r(x, a)+V_M(x') \in \mathcal{F}$である場合に常に成立します．実際，この場合はMisfitが平均ベルマン誤差に一致します．ただし，必ずしもこの場合を含む必要はありません．

---


## Witness rank

これまではWitness model misfitを定義し，仮定を一つ導入しました．実はこれだけでは十分ではありません．指数的なサンプルが必要になるMDPが存在してしまいます（論文のProposition 4）．

そこで，Witness rankの仮定を導入します．ノリはBellman rankと同じです．

---

**Witness rank**

モデルクラス$\mathcal{M}$, テスト関数$\mathcal{F}$，そして$\kappa \in (0, 1]$を考えます．
それぞれの$h \in [H]$に対して，行列の集合$\mathcal{N}_{\kappa, h}$を次のように定めます．
任意の$A \in \mathcal{N}_{\kappa, h}$が次を満たす：

$$
A \in \mathbb{R}^{|\mathcal{M}| \times|\mathcal{M}|}, \quad \kappa \mathcal{E}_B\left(M, M^{\prime}, h\right) \leq A\left(M, M^{\prime}\right) \leq \mathcal{W}\left(M, M^{\prime}, h\right), \forall M, M^{\prime} \in \mathcal{M},
$$

これについて，Witness rankを

$$
\mathrm{W}(\kappa, \beta, \mathcal{M}, \mathcal{F}, h) \triangleq \min _{A \in \mathcal{N}_{\kappa, h}} \operatorname{rank}(A, \beta)
$$

として定義します．

**解釈**

* $A(M, M')={W}\left(M, M^{\prime}, h\right)$である場合を考えましょう．
この時，$0$でないwitnessed model misfitをチェックするためにかかる文脈の分布の数はWitness rankで抑えられます．よって，$M^\star$を見つけるためにかかる時間はwitness rankで抑えられます．
* 一方で，$A\left(M, M^{\prime}\right)=\kappa \mathcal{E}_B\left(M, M^{\prime}, h\right)$の場合はこれはBellman rankと同じです．よって，上のWitness rankの定義はBellman rankを一般化してることがわかります．


## アルゴリズム

Model misfitを推定して最適方策を出すアルゴリズムを見てみましょう．

データセット$\mathcal{D}=\{(x_h^{(n)}, a_h^{(n)}, r_h^{(n)}, x_{h+1}^{(n)})\}_{n=1}^N$が与えられたときを考えます．ここで，

$$
x_h^{(n)} \sim \pi_M, a_h^{(n)} \sim U(\mathcal{A}), (r_h^{(n)}, x_{h+1}^{(n)}) \sim M_h^\star
$$

です．また，importance weightを$\rho^{(n)} = K\pi_{M'}(a_h^{(n)} \mid x_h^{(n)})$とします．

つまり，
* 各ステップ$h$までは$\pi_M$で動き，$x_h^{(n)}$を出す
* その次に行動$a_h^{(n)}$を$U(\mathcal{A})$から出す
* 報酬と次状態は真のMDPからサンプルする

状況です．これを使ってmisfitを推定してみましょう．

$$
\widehat{\mathcal{W}}(M, M', h) = \max_{f \in \mathcal{F}}
\sum^{N}_{n=1}\frac{\rho^{(n)}}{N}
\mathbb{E}_{(r, x')\sim M'_h}
\left[f(x_h^{(n)}, a_h^{(n)}, r, x') - f(x_h^{(n)}, a_h^{(n)}, r^{(n)}_h, x^{(n)}_{h+1})\right]
$$

ここで，行動は$U(\mathcal{A})$からサンプルしてるので，重み$\rho^{n}$で修正しています．

これは通常の一様収束の話を使えば$\mathcal{W}(M, M', h)$に収束することが言えます．

また，平均ベルマン誤差$\mathcal{E}_B(M, M, h)$も推定します．
データセット
$\{(x_h^{(n)}, a_h^{(n)}, r_h^{(n)}, x_{h+1}^{(n)})\}_{n=1}^N$
を考えます．ここで，
$$
x_h^{(n)} \sim \pi_M, a_h^{(n)} \sim \pi_M, (r_h^{(n)}, x_{h+1}^{(n)}) \sim M_h^\star
$$
を使って，unbiasedな推定
$$
\widehat{\mathcal{E}}_B(M, M, h) = \frac{1}{N} \sum^N_{n=1} 
\left[Q_M(x^{(n)}_h, a^{(n)}_h) - \left[r_h^{(n)} + V_M(x_{h+1}^{(n)})\right]\right]
$$
をします．

以上を踏まえて，次のアルゴリズムを考えます：

![witness-rank](figs/Witness-rank.png)


このアルゴリズムでは，ループ毎に$\mathcal{M}_t$から最適なモデル以外が除去されていきます．

* ３行目ではOptimisticなモデルを選択し，４行目でその価値をサンプルから推定してます．
* ４，５行目で，その推定値がOptimisticなモデルの価値とあまり変わらない場合，その方策をそのまま返します．
* ６行目で，平均ベルマン誤差が高いステップを探します．
* ７行目で，このステップの精度を上げます
* 最後にWitness misfitが一定以上のモデルを排除します．$M^\star$のmisfitは０なので

## サンプル効率の導出

次が成立します．任意の$\epsilon, \delta, \kappa \in (0, 1]$について，$\phi=\frac{\kappa \epsilon}{48 H \sqrt{\mathrm{W}_\kappa}}$および$T=H \mathrm{~W}_\kappa \log (\beta / 2 \phi) / \log (5 / 3)$を考えます．
また，
$n_e=\Theta\left(H^2 \log (H T / \delta) / \epsilon^2\right)$ and $n=\Theta\left(H^2 K \mathrm{~W}_\kappa \log (T|\mathcal{M} \| \mathcal{F}| / \delta) /(\kappa \epsilon)^2\right)$
として，Algorithm 1を実行すると，確率$1-\delta$以上で，$\epsilon$-最適な方策が返り，それに必要な軌跡の数は

$$
\tilde{O}\left(\frac{H^3 \mathrm{~W}_\kappa^2 K}{\kappa^2 \epsilon^2} \log \left(\frac{T|\mathcal{F}||\mathcal{M}|}{\delta}\right)\right)
$$

です．

---

これは有限な$|\mathcal{F}|$についてのみの話ですが，無限の場合でも拡張できます（Theorem 7を確認してください）．
有限な$|\mathcal{F}|$についての証明をしましょう．

**証明**

まず，いくつかの便利な補題について確認しましょう．
平均ベルマン誤差やWitness model misfitなどの量は簡単にHoeffdingでバウンドできるので，先に終わらせてしまいます．
また，
任意のモデル$M$, $M'$について，平均ベルマン誤差は

$$
\begin{aligned}
& \mathcal{E}_B\left(M, M^{\prime}, h\right) \triangleq \mathcal{E}_B\left(Q_M, Q_{M^{\prime}}, h\right) \\
= & \mathbb{E}_{x_h \sim \pi_M, a_h \sim \pi_{M^{\prime}}}\left[\mathbb{E}_{\left(r, x^{\prime}\right) \sim M_h^{\prime}}\left[r+V_{M^{\prime}}\left(x^{\prime}\right)\right]-\mathbb{E}_{\left(r, x^{\prime}\right) \sim M_h^{\star}}\left[r+V_{M^{\prime}}\left(x^{\prime}\right)\right]\right]
\end{aligned}
$$

の形でかけることに注意しましょう．

---



**補題（$|v^\pi - v^\pi|$）**

固定された方策$\pi$と$\epsilon, \delta \in (0, 1)$を考えます．
$n_e=\frac{\log (2 / \delta)}{(2 \epsilon)^2}$ 個の$\pi$によって収集された軌跡$\left\{\left(x_h^{(i)}, a_h^{(i)}, r_h^{(i)}\right)_{h=1}^H\right\}_{i=1}^{n_e}$
を考え，$\hat{v}^\pi=\frac{1}{n_e} \sum_{i=1}^{n_e} \sum_{h=1}^H r_h^{(i)}$とします．
すると，確率$1-\delta$以上で
$\left|\hat{v}^\pi-v^\pi\right| \leq \epsilon$です．

証明は単純なHoeffdingの活用なので省略します．

---

**補題（$|\widetilde{\mathcal{W}}-\mathcal{W}|$）**

$h$と$M\in \mathcal{M}$を固定します．
データセット

$$
\mathcal{D}=\left\{\left(x_h^{(i)}, a_h^{(i)}, r_h^{(i)}, x_{h+1}^{(i)}\right)\right\}_{i=1}^N \text { with } x_h^{(i)} \sim \pi_M, a_h^{(i)} \sim U(\mathcal{A}),\left(r_h^{(i)}, x_{h+1}^{(i)}\right) \sim M_h^{\star}
$$

をサンプルします．このとき，確率$1-\delta$以上で，任意の$M'\in \mathcal{M}$について，

$$
\left|\widehat{\mathcal{W}}\left(M, M^{\prime}, h\right)-\mathcal{W}\left(M, M^{\prime}, h\right)\right| \leq \sqrt{\frac{2 K \log (2|\mathcal{M}||\mathcal{F}| / \delta)}{N}}+\frac{2 K \log (2|\mathcal{M} \| \mathcal{F}| / \delta)}{3 N}
$$

が成立します．

**証明**

定義を思い出しましょう．


$$
\widehat{\mathcal{W}}(M, M', h) = \max_{f \in \mathcal{F}}
\sum^{N}_{n=1}\frac{\rho^{(n)}}{N}
\mathbb{E}_{(r, x')\sim M'_h}
\left[f(x_h^{(n)}, a_h^{(n)}, r, x') - f(x_h^{(n)}, a_h^{(n)}, r^{(n)}_h, x^{(n)}_{h+1})\right]
$$
および
$$
\mathcal{W}\left(M, M^{\prime}, h ; \mathcal{F}\right) \triangleq \sup _{f \in \mathcal{F}} \underset{\substack{x_h \sim \pi_M \\ a_h \sim \pi_{M^{\prime}}}}{\mathbb{E}}\left[\underset{\left(r, x^{\prime}\right) \sim M_h^{\prime}}{\mathbb{E}}\left[f\left(x_h, a_h, r, x^{\prime}\right)\right]-\underset{\left(r, x^{\prime}\right) \sim M_h^{\star}}{\mathbb{E}}\left[f\left(x_h, a_h, r, x^{\prime}\right)\right]\right],
$$
でした．


$M' \in \mathcal{M}$と$f\in \mathcal{F}$を固定します．また，次の確率変数を定義します（$K=|\mathcal{A}|$です．$a$が一様分布から出てるので，importance weight用です．）

$$
z_i\left(M^{\prime}, f\right) \triangleq K \pi_{M^{\prime}}\left(a_h^{(i)} \mid x_h^{(i)}\right)\left(\underset{\left(r, x^{\prime}\right) \sim M_h^{\prime}}{\mathbb{E}} f\left(x_h^{(i)}, a_h^{(i)}, r, x^{\prime}\right)-f\left(x_h^{(i)}, a_h^{(i)}, r_h^{(i)}, x_{h+1}^{(i)}\right)\right)
$$

このとき，$z_i(M', f)$の期待値は

$$
\mathbb{E}\left[z_i\left(M^{\prime}, f\right)\right]=\underbrace{\underset{x_h \sim \pi_M, a_h \sim \pi_{M^{\prime}}}{\mathbb{E}}\left[\underset{\left(r, x^{\prime}\right) \sim M_h^{\prime}}{\mathbb{E}}\left[f\left(x_h, a_h, r, x^{\prime}\right)\right]-\underset{\left(r, x^{\prime}\right) \sim M_h^{\star}}{\mathbb{E}}\left[f\left(x_h, a_h, r, x^{\prime}\right)\right]\right]}_{\triangleq d\left(M^{\prime}, M^{\star}, f\right)},
$$

になります（これは$\mathcal{W}$の中身ですね）．このとき，$\|f\|_\infty \leq 2$の仮定から，$\operatorname{Var}\left(z_i\left(M^{\prime}, f\right)\right) \leq 4 K$であることがすぐにわかります．

Bernsteinを使うと，確率$1-\delta$以上で

$$
\left|\frac{1}{N} \sum_{i=1}^N z_i\left(M^{\prime}, f\right)-d\left(M^{\prime}, M^{\star}, f\right)\right| \leq \sqrt{\frac{2 K \log (2 / \delta)}{N}}+\frac{2 K \log (2 / \delta)}{3 N}
$$

が成立します．Union boundをとれば，任意の$M' \in \mathcal{M}$と$f \in \mathcal{F}$について，

$$
\left|\frac{1}{N} \sum_{i=1}^N z_i\left(M^{\prime}, f\right)-d\left(M^{\prime}, M^{\star}, f\right)\right| \leq \sqrt{\frac{2 K \log (2|\mathcal{M}||\mathcal{F}| / \delta)}{N}}+\frac{2 K \log (2|\mathcal{M} \| \mathcal{F}| / \delta)}{3 N}
$$

が確率$1-\delta$以上で成立することがわかります．
これは任意の$f \in \mathcal{F}$について成立するので，$\max_{f \in \mathcal{F}} 経験$と$\sup_{f \in \mathcal{F}} \mathbb{E}$もバウンドされます（TODO: ちゃんと確かめる）．

---

**補題（$|\mathcal{E}_B - \widehat{\mathcal{E}}_B|$）**

$x_h^{(i)} \sim \pi_M, a_h^{(i)} \sim \pi_M,\left(r_h^{(i)}, x_{h+1}^{(i)}\right) \sim M_h^{\star}$についてのデータセット$\mathcal{D}=\left\{\left(x_h^{(i)}, a_h^{(i)}, r_h^{(i)}, x_{h+1}^{(i)}\right)\right\}_{i=1}^N$について，
確率$1-\delta$以上で

$$
\left|\mathcal{E}_B(M, M, h)-\widehat{\mathcal{E}}_B(M, M, h)\right| \leq \sqrt{\frac{\log (2 H / \delta)}{2 N}} .
$$

が成立します．証明はHoeffdingからすぐにわかります．



---

**補題：Bellman rankの補題11**

closedかつboundedな集合$V\subset \mathbb{R}^d$とベクトル$v \in \mathbb{R}^d$を考えます．
$B$を中心が$(0, 0, \dots)$であり，$V$を含む楕円とします．
$p^\top v \geq \kappa$を満たすような$v\in V$が存在するとし，$B_+$を
$\left\{v \in B:\left|p^{\top} v\right| \leq \frac{\kappa}{3 \sqrt{d}}\right\}$
の最小楕円とします．
このとき，

$$
\frac{\operatorname{vol}\left(B_{+}\right)}{\operatorname{vol}(B)} \leq \frac{3}{5}
$$
が成立します．

---

**補題（Simulation Lemma）**

Witness rankが成立するとき，

$$
v_M-v^{\pi_M}=\sum_{h=1}^H \mathcal{E}_B(M, M, h), \quad \text { and } \quad v_M-v^{\pi_M} \leq \sum_{h=1}^H \mathcal{W}(M, M, h)
$$

が成り立ちます．

**証明**

$h=1$のとき，

$$
\begin{aligned}
& \mathbb{E}_{x_1 \sim P_0}\left[V_M\left(x_1\right)-V^{\pi_M}\left(x_1\right)\right] \\
& =\mathbb{E}_{x_1 \sim P_0, a_1 \sim \pi_M}\left[\mathbb{E}_{\left(r, x_2\right) \sim M_{x_1, a_1}}\left[r+V_M\left(x_2\right)\right]-\mathbb{E}_{\left(r, x_2\right) \sim M_{x_1, a_1}^{\star}}\left[r+V^{\pi_M}\left(x_2\right)\right]\right] \\
& =\mathbb{E}_{x_1 \sim P_0, a_1 \sim \pi_M}\left[\mathbb{E}_{\left(r, x_2\right) \sim M_{x_1, a_1}}\left[r+V_M\left(x_2\right)\right]-\mathbb{E}_{\left(r, x_2\right) \sim M_{x_1, a_1}^{\star}}\left[r+V_M\left(x_2\right)\right]\right] \\
& \quad+\mathbb{E}_{x_1 \sim P_0, a_1 \sim \pi_M}\left[\mathbb{E}_{\left(r, x_2\right) \sim M_{x_1, a_1}^{\star}}\left[V_M\left(x_2\right)\right]-\mathbb{E}_{\left(r, x_2\right) \sim M_{x_1, a_1}^{\star}}\left[V^{\pi_M}\left(x_2\right)\right]\right],
\end{aligned}
$$

が成り立ちます．この一項目は$\mathcal{E}_B(M, M, 1)$と同じであることに注意しましょう．また，二項目は

$$
\mathbb{E}\left[V_M\left(x_2\right)-V^{\pi_M}\left(x_2\right) \mid x_2 \sim \pi_M\right]
$$

になります．あとは再帰的に分解していけば題意の１つ目を得ます．
また，$V_M \in \mathcal{F}$であることを踏まえると，$\mathcal{E}_B(M, M, h) \leq \mathcal{W}(M, M, h)$であることがすぐにわかります．これで２つ目も証明完了です．

---



**補題**

$$
\hat{v}^{\pi^t} \text { satisfies }\left|v^{\pi^t}-\hat{v}^{\pi^t}\right| \leq \epsilon / 8
$$
が成立しているラウンド$t$を考えます（$M^\star$は除外されてないとします）．

このとき，次のどちらかが成立します：

1. アルゴリズムは終了せず，$\mathcal{E}_B\left(M^t, M^t, h\right) \geq \frac{3 \epsilon}{8 H}$なるステップ$h$が存在する
2. アルゴリズムは終了し，$v^{\pi^t} \geq v^{\star}-\epsilon$を吐き出す．

証明は省略（簡単）．

---


